##### **IMPORTATION DES FONCTIONS**

In [1]:
from config import Config
from utils import load_matrices, compute_infra_tensor
from access_index_v2 import compute_access_index
from remoteness_index import compute_remoteness_index
from simulations import simulate_scenario

##### **CHARGEMENT DES MATRICES**

In [2]:
## Chargement des bases de données
distance_tensor, infra_tensor, population_tensor = load_matrices(path_dt = Config.PATH_DT_MATRIX,
                                                                    path_pop = Config.PATH_POP_MATRIX,
                                                                    path_infra = Config.PATH_INFRA_MATRIX,
                                                                    device = Config.DEVICE)

In [3]:
## Choix des secteurs pour le calcul de l'accessibilité
## Ici le secteur 06 est choisi (éducation)

infra_tensor = compute_infra_tensor(infra_tensor=infra_tensor,
                                    prefix=["06"])

In [4]:
## Calcul des indices d'éloignement
remoteness_index = compute_remoteness_index(distance_matrix=distance_tensor, 
                                            population_vector=population_tensor, 
                                            infrastructure_matrix=infra_tensor)

INFO:remoteness_index:Calcul de la distance minimale vers chaque type d'infrastructure...
INFO:remoteness_index:Calcul de la moyenne pondérée par la population (en excluant les valeurs nulles)...
INFO:remoteness_index:Normalisation des distances par les moyennes...
INFO:remoteness_index:Tronquage des distances normalisées au seuil 3.0...


In [5]:
## Calcul des indices d'accessibilité
access_index = compute_access_index(distance_tensor=distance_tensor, 
                                    population_tensor=population_tensor, 
                                    infra_tensor=infra_tensor, 
                                    device=Config.DEVICE)

INFO:access_index_v2:Construction du tenseur de distance pondérée D[i, j, t]...
INFO:access_index_v2:Calcul des distances maximales par type d'infrastructure...
INFO:access_index_v2:Normalisation des distances...
INFO:access_index_v2:Application de la fonction de décroissance exponentielle...
INFO:access_index_v2:Calcul de la probabilité de parcours des distances selon l'offre...
INFO:access_index_v2:Calcul du dénominateur (somme pondérée inversée)...
INFO:access_index_v2:Calcul du numérateur (produit pondéré)...
INFO:access_index_v2:Calcul final de l'indice d'accessibilité...


In [6]:
access_index

LabelledTensor(shape=torch.Size([8396, 15]), dims=['i', 't'])

In [8]:
access_index.to_dataframe(
    index_dim="i",
    column_dim="t"
).describe()

statistic,i,06,06001,060010001,060010002,060010003,060010004,06002,060020001,060020002,060020004,06003,060030001,060030002,060030004,060030005
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""8396""",8396.0,8396.0,8396.0,8396.0,8396.0,8396.0,8396.0,8396.0,8396.0,8396.0,8396.0,8396.0,8396.0,8396.0,8396.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,2.084898,2.028232,0.006829,1.667363,0.041043,0.000169,0.000938,0.000766,0.000031,4.6732e-8,0.001186,4.7075e-8,0.001135,4.3383e-8,0.000001
"""std""",null,0.598986,0.575431,0.000454,0.443581,0.004281,0.000004,0.000038,0.00003,4.9227e-7,6.1926e-12,0.000051,6.5718e-12,0.000049,3.8543e-12,4.8032e-9
"""min""","""010020201001""",0.007852,0.007598,0.000298,0.006132,0.001043,0.000052,0.000119,0.000102,0.000012,4.6407e-8,0.000113,4.6728e-8,0.000109,4.3213e-8,8.6273e-7
"""25%""",null,1.756254,1.717072,0.006837,1.447159,0.040647,0.000169,0.00094,0.000768,0.000031,4.6732e-8,0.001189,4.7075e-8,0.001138,4.3384e-8,0.000001
"""50%""",null,2.201506,2.143349,0.00694,1.76866,0.042328,0.000169,0.000945,0.000772,0.000031,4.6733e-8,0.001196,4.7075e-8,0.001145,4.3384e-8,0.000001
"""75%""",null,2.540199,2.464618,0.006994,1.99882,0.043246,0.00017,0.000948,0.000774,0.000031,4.6733e-8,0.001199,4.7075e-8,0.001148,4.3384e-8,0.000001
"""max""","""330840498011""",2.983845,2.881422,0.007045,2.282181,0.044153,0.00017,0.00095,0.000775,0.000031,4.6733e-8,0.001203,4.7076e-8,0.001151,4.3384e-8,0.000001


##### **AFFICHAGE DES INDICES**

In [7]:
## Affichage des indices d'éloignement 
## sous forme de tableau
remoteness_index.to_dataframe(
    index_dim="i",
    column_dim="t"
)

i,06,06001,060010001,060010002,060010003,060010004,06002,060020001,060020002,060020004,06003,060030001,060030002,060030004,060030005
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""010020201001""",0.0,0.0,0.077681,0.0,0.148935,0.01893,0.048651,0.04736,0.111639,0.096283,0.228878,0.069117,0.228756,0.949187,0.237089
"""010020201003""",0.0,0.0,0.0,0.0,1.038956,0.179055,0.339382,0.330379,0.376711,0.126974,0.543678,0.128609,0.543389,0.918546,0.352671
"""010020201004""",0.0,0.0,0.0,0.0,0.0,0.107774,0.133011,0.129482,0.270709,0.085534,0.502502,0.088373,0.502235,0.889188,0.308092
"""010020201005""",0.0,0.0,0.0,0.0,0.0,0.056019,0.0,0.0,0.196481,0.069694,0.289423,0.071714,0.28927,0.873639,0.275726
"""010020201007""",0.0,0.0,0.267913,0.0,0.513658,0.132861,0.16779,0.163339,0.286598,0.088924,0.447184,0.096449,0.446946,0.893253,0.323782
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""330840498007""",0.0,0.0,0.960488,0.0,0.881385,0.448334,0.60154,0.585581,0.758155,0.364381,0.963644,0.386164,0.963132,0.673078,0.146376
"""330840498008""",0.0,0.0,0.252738,0.0,0.349789,0.278282,0.158286,0.154087,0.400095,0.339705,0.253568,0.360211,0.253433,0.621988,0.038517
"""330840498009""",0.0,0.0,1.422294,0.0,1.039297,0.832396,0.892063,0.868397,0.497826,0.308829,1.426969,0.327737,1.426211,0.788464,0.386562


In [28]:
## Affichage des indices d'accessibilité
## sous forme de tableau
access_index.display()

i,t,valeur
null,null,null


##### **SIMULATIONS**

In [18]:
## DEFINITION DES PARAMETRES DE SIMULATIONS
## Ceci sont pour les modifications appliquées
## des localités spécifiques et des secteurs spécifiques
MODIFICATIONS_TRONCONS = {"21337": "RB", "46371": "RB", "45886": "RB"}
MODIFICATIONS_INFRAS = {"010020201001": {"060010001": 2}, 
                        "010020201003": {"060010002": 2, "060010003": 4}}

MODIFICATION_POP = {"010020201001": 0.06, 
                    "010020201003": 0.05,
                    "330840498007": 0.09}

In [19]:
## DETERMINATIONS DES NOUVELLES MATRICES APRES
## MODIFICATIONS DES PARAMETRES
sim_mat_dt, sim_mat_infra, sim_mat_pop = simulate_scenario(gid_modifications = MODIFICATIONS_TRONCONS,
                                        voie_type_global_change = None,
                                        infra_modifications = MODIFICATIONS_INFRAS,
                                        population_growth_rates = MODIFICATION_POP,
                                        global_population_growth = None,
                                        path_mat_path = Config.PATH_ITINERAIRE_MATRIX,
                                        path_mat_infra = Config.PATH_INFRA_MATRIX,
                                        path_mat_distance_temps = Config.PATH_DT_MATRIX,
                                        path_mat_population = Config.PATH_POP_MATRIX,
                                        path_geometrie_troncons = Config.PATH_GEOMETRIE_TRONCONS
                                        )

INFO:simulations:Début de la simulation de scénario de transport...
INFO:simulations:Application des changements spécifiques de type de voie par gid...
INFO:simulations:Recalcul des temps de parcours depuis la matrice unique...


3 tronçon(s) affecté(s) identifié(s).


INFO:simulations:Mise à jour des infrastructures locales...
INFO:simulations:Application de taux de croissance spécifiques à certaines localités...
INFO:simulations:Création des tenseurs labéllisés...
INFO:simulations:Création des tenseurs de simulation...


In [20]:
## Calcul des indices d'accessibilité et d'éloignement
infra_tensor = compute_infra_tensor(infra_tensor=sim_mat_infra,
                                    prefix=["06"])
remoteness_index = compute_remoteness_index(distance_matrix=sim_mat_dt, 
                                            population_vector=sim_mat_pop, 
                                            infrastructure_matrix=infra_tensor)
access_index = compute_access_index(distance_tensor=sim_mat_dt, 
                                    population_tensor=sim_mat_pop,
                                    infra_tensor=infra_tensor,
                                    device=Config.DEVICE)

INFO:remoteness_index:Calcul de la distance minimale vers chaque type d'infrastructure...
INFO:remoteness_index:Calcul de la moyenne pondérée par la population (en excluant les valeurs nulles)...
INFO:remoteness_index:Normalisation des distances par les moyennes...
INFO:remoteness_index:Tronquage des distances normalisées au seuil 3.0...
INFO:access_index:Construction du tenseur de distance pondérée D[i, j, t]...
INFO:access_index:Calcul des distances maximales par type d'infrastructure...
INFO:access_index:Normalisation des distances...
INFO:access_index:Application de la fonction de décroissance exponentielle...
INFO:access_index:Restriction aux infrastructures réellement présentes...
INFO:access_index:Calcul du dénominateur (somme pondérée inversée)...
INFO:access_index:Calcul du numérateur (produit pondéré)...
INFO:access_index:Calcul final de l'indice d'accessibilité...
